In [1]:
from langchain_core.runnables import RunnableLambda


paso1 = RunnableLambda(lambda x: f"Numero {x}")

def duplicar_text(texto):
    return [texto]*2

paso2 = RunnableLambda(duplicar_text)

cadena = paso1 | paso2

resultado = cadena.invoke(43)
print(resultado)

['Numero 43', 'Numero 43']


#### Mini-Proyecto

In [3]:
# 1. Configuración Inicial
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_openai import ChatOpenAI
import json

llm = ChatOpenAI(model='gpt-4o-mini', temperature='0')

In [4]:
# 2. Preprocesador de Texto

def preprocess_text(text):
    """Limpia el texto eliminando espacios extras y limitando longitud"""
    # Pista: usa .strip() para eliminar espacios
    # Pista: limita a 500 caracteres con slicing [:500]
    return text.strip()[:500] # ¡Completa aquí!
 
# Convertir la función en un Runnable
preprocessor = RunnableLambda(preprocess_text)


In [5]:
#3. Generador de Resúmenes
def generate_summary(text):
    """Genera un resumen conciso del texto"""
    prompt = f"""Resume en una sola oración considerando la idea
    principal del texto. Maneja oraciones de menos de 500 palabras : {text}"""
    response = llm.invoke(prompt)
    return response.content

summary_branch = RunnableLambda(generate_summary)

In [6]:
# 4. Analizador de Sentimientos

def analyze_sentiment(text):
    """Analiza el sentimiento y devuelve resultado estructurado"""
    prompt = f"""Analiza el sentimiento del siguiente texto.
    Responde ÚNICAMENTE en formato JSON válido:
    {{"sentimiento": "positivo|negativo|neutro", "razon": "justificación breve"}}
    
    Texto: {text}"""
    
    response = llm.invoke(prompt)
    try:
        return json.loads(response.content)
    except json.JSONDecodeError:
        return {"sentimiento": "neutro", "razon": "Error en análisis"}
    
sentiment_branch = RunnableLambda(analyze_sentiment)

In [7]:
# 5. Función de Combinación

def merge_results(data):
    """Combina los resultados de ambas ramas en un formato unificado"""
    return {
        "resumen": data["resumen"],
        "sentimiento": data["sentimiento_data"]["sentimiento"],
        "razon": data["sentimiento_data"]["razon"]
    }

merger = RunnableLambda(merge_results)

In [8]:
# 6. Función de Procesamiento Principal

paralel_analysis = RunnableParallel({
    "resumen":summary_branch,
    "sentimiento_data":sentiment_branch
})
 

In [9]:
# 7. Construcción de la Cadena Final
chain = preprocessor | paralel_analysis | merger

In [10]:
# Prueba con diferentes textos
textos_prueba = [
    "¡Me encanta este producto! Funciona perfectamente y llegó muy rápido.",
    "El servicio al cliente fue terrible, nadie me ayudó con mi problema.",
    "El clima está nublado hoy, probablemente llueva más tarde."
]
 
for texto in textos_prueba:
    resultado = chain.invoke(texto)
    print(f"Texto: {texto}")
    print(f"Resultado: {resultado}")
    print("-" * 50)

Texto: ¡Me encanta este producto! Funciona perfectamente y llegó muy rápido.
Resultado: {'resumen': 'Este producto es excelente, ya que funciona a la perfección y llegó rápidamente.', 'sentimiento': 'positivo', 'razon': 'El usuario expresa satisfacción con el producto y destaca su buen funcionamiento y rapidez en la entrega.'}
--------------------------------------------------
Texto: El servicio al cliente fue terrible, nadie me ayudó con mi problema.
Resultado: {'resumen': 'El servicio al cliente fue deficiente, ya que no recibí la ayuda necesaria para resolver mi problema.', 'sentimiento': 'negativo', 'razon': 'El texto expresa insatisfacción con el servicio al cliente y menciona que no recibió ayuda.'}
--------------------------------------------------
Texto: El clima está nublado hoy, probablemente llueva más tarde.
Resultado: {'resumen': 'Hoy el clima es nublado y es probable que llueva más tarde.', 'sentimiento': 'neutro', 'razon': 'El texto describe una condición climática sin e

In [11]:
resultado_batch = chain.batch(textos_prueba)
print(resultado_batch)

[{'resumen': 'El producto es excelente, funciona a la perfección y llegó rápidamente.', 'sentimiento': 'positivo', 'razon': 'El usuario expresa satisfacción y aprecio por el producto y su entrega.'}, {'resumen': 'El servicio al cliente fue deficiente, ya que no recibí la ayuda necesaria para resolver mi problema.', 'sentimiento': 'negativo', 'razon': 'El texto expresa insatisfacción con el servicio al cliente, indicando que no recibió la ayuda necesaria.'}, {'resumen': 'Hoy el clima es nublado y es probable que llueva más tarde.', 'sentimiento': 'neutro', 'razon': 'El texto describe una condición climática sin expresar emociones positivas o negativas.'}]


#### Probar Prompt Templates

In [12]:
from langchain_core.prompts import PromptTemplate

template = " Eres un experto de marketing. Sugiere un eslogan  creativo para un producto {producto}"

prompt = PromptTemplate(
    template=template,
    input_variables=["producto"]
)

prompt_lleno = prompt.format(producto = "café organico")
print(prompt_lleno)

 Eres un experto de marketing. Sugiere un eslogan  creativo para un producto café organico


#### Usando ChatPromptTemplate

In [16]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system","Eres un traductor del español al inglés muy precios."),
    ("human","{texto}")
])

mensajes = chat_prompt.format_messages(texto="Hola mundo, ¿cómo estás?")

for m in mensajes:
    print(f"{type(m)}: {m.content}")

<class 'langchain_core.messages.system.SystemMessage'>: Eres un traductor del español al inglés muy precios.
<class 'langchain_core.messages.human.HumanMessage'>: Hola mundo, ¿cómo estás?


#### Message Placeholder

In [19]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente util que mantiene el contexto de la conversación"),
    MessagesPlaceholder(variable_name="historial"),
    ("human","Usuario: {pregunta_actual}")
])
historial_conversacion = [
    HumanMessage(content="Usuario: ¿Cual es la capital de Francia?"),
    AIMessage(content="IA: La capital de Francia es París."),
    HumanMessage(content="Usuario: ¿Y cuantos habitantes tiene?"),
    AIMessage(content="IA: Paris tiene aproximadamente 2.2 millones de habitantes en la ciudad."),
]

mensajes = chat_prompt.format_messages(
    historial = historial_conversacion,
    pregunta_actual = "¿Puedes contarme algo sobre su arquitectura?"
)

for m in mensajes:
    print(m.content)

Eres un asistente util que mantiene el contexto de la conversación
Usuario: ¿Cual es la capital de Francia?
IA: La capital de Francia es París.
Usuario: ¿Y cuantos habitantes tiene?
IA: Paris tiene aproximadamente 2.2 millones de habitantes en la ciudad.
Usuario: ¿Puedes contarme algo sobre su arquitectura?


#### Usando Few-Shot
Es una técnica de In-Context Learning donde proporcionamos al modelo entre 2-10 ejemplos de la tarea que queremos que realice. Estos ejemplos sirven como "entrenamiento instantáneo".

Mejores Prácticas

- 2-5 ejemplos: Suficiente para mostrar el patrón, no demasiados

- Ejemplos diversos: Cubrir diferentes casos/variaciones

- Formato consistente: Mismo patrón en todos los ejemplos

- Ejemplos de calidad: Outputs perfectos como referencia

MessagesPlaceholder transforma los few-shot examples de una técnica básica a una herramienta poderosa y flexible. La estructura clara de mensajes Human/AI hace que el modelo comprenda mejor qué esperamos, resultando en respuestas más precisas y consistentes.

In [20]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
 
# Template para clasificación de sentimientos con few-shot examples
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un experto en análisis de sentimientos. Clasifica cada texto como: POSITIVO, NEGATIVO o NEUTRO."),
    MessagesPlaceholder(variable_name="ejemplos"),
    ("human", "Texto a analizar: {texto_usuario}")
])
 
# Few-shot examples para análisis de sentimientos
ejemplos_sentimientos = [
    HumanMessage(content="Texto a analizar: Me encanta este producto, es increíble"),
    AIMessage(content="POSITIVO"),
    HumanMessage(content="Texto a analizar: El servicio fue terrible, muy decepcionante"),
    AIMessage(content="NEGATIVO"),
    HumanMessage(content="Texto a analizar: El clima está nublado hoy"),
    AIMessage(content="NEUTRO")
]
 
# Generar el prompt con los ejemplos
mensajes = chat_prompt.format_messages(
    ejemplos=ejemplos_sentimientos,
    texto_usuario="¡Qué día tan maravilloso!"
)
 
# Ver el resultado
for i, m in enumerate(mensajes):
    print(f"Mensaje {i+1} ({m.__class__.__name__}):")
    print(m.content)
    print("-" * 40)

Mensaje 1 (SystemMessage):
Eres un experto en análisis de sentimientos. Clasifica cada texto como: POSITIVO, NEGATIVO o NEUTRO.
----------------------------------------
Mensaje 2 (HumanMessage):
Texto a analizar: Me encanta este producto, es increíble
----------------------------------------
Mensaje 3 (AIMessage):
POSITIVO
----------------------------------------
Mensaje 4 (HumanMessage):
Texto a analizar: El servicio fue terrible, muy decepcionante
----------------------------------------
Mensaje 5 (AIMessage):
NEGATIVO
----------------------------------------
Mensaje 6 (HumanMessage):
Texto a analizar: El clima está nublado hoy
----------------------------------------
Mensaje 7 (AIMessage):
NEUTRO
----------------------------------------
Mensaje 8 (HumanMessage):
Texto a analizar: ¡Qué día tan maravilloso!
----------------------------------------


#### Rol Prompt Templates

In [3]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

plantilla_sistema = SystemMessagePromptTemplate.from_template("" \
"Eres un {rol} especializado en {especialidad}. Responde de manera {tono}"
)

plantilla_humano = HumanMessagePromptTemplate.from_template(
    "Mi pregunta sobre {tema} es: {pregunta}"
)

chat_prompt = ChatPromptTemplate.from_messages([
    plantilla_sistema,
    plantilla_humano
])

mensajes = chat_prompt.format_messages(
    rol = "nutricionista",
    especialidad = "dietas veganas",
    tono = "profesional pero accesible",
    tema= "proteina vegetales",
    pregunta="¿Cuales son las mejores fuentes de proteinas veganas para un atleta profesional?"
)

for m in mensajes:
    print(m.content)

Eres un nutricionista especializado en dietas veganas. Responde de manera profesional pero accesible
Mi pregunta sobre proteina vegetales es: ¿Cuales son las mejores fuentes de proteinas veganas para un atleta profesional?


#### Uso Pydantic

In [7]:
from pydantic import BaseModel

class Usuario(BaseModel):
    id: int
    nombre: str
    activo: bool = True


data = {"id": "123", "nombre":"Ana"}

usuario = Usuario(**data)

print(usuario.model_dump_json())

{"id":123,"nombre":"Ana","activo":true}


#### Output parses en Langchain

In [11]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class AnalisisTexto(BaseModel):
    resumen: str = Field(description="Resumen breve del texto.")
    sentimientos: str = Field(description="Sentimientos del texto (Positivo, neutro o negativo")

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.6)

structured_llm = llm.with_structured_output(AnalisisTexto)

texto_prueba = "Me encantó la nueva pelicula de acción, tiene muchos efectos especiales"

resultado = structured_llm.invoke(f'Analiza el siguiente exto: {texto_prueba}')

print(resultado.model_dump_json())

{"resumen":"El hablante expresa su entusiasmo por una nueva película de acción, destacando sus efectos especiales.","sentimientos":"Positivo"}


#### Output parses antiguos

In [13]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

 
class AnalisisTexto(BaseModel):
    resumen: str = Field(description="Resumen breve del texto")
    sentimiento: str = Field(description="Sentimiento del texto (Positivo, neutro o negativo)")
    palabras_clave: list[str] = Field(description="Lista de palabras clave del texto")

# Crear el parser con nuestro modelo
parser = PydanticOutputParser(pydantic_object=AnalisisTexto)
 
# Ver las instrucciones que genera automáticamente
print("Instrucciones generadas:")
print(parser.get_format_instructions())

prompt = PromptTemplate(
    template="""Eres un experto analista de texto. Analiza el siguiente texto con mucho cuidado y proporciona un análisis detallado.
 
{format_instructions}
 
Texto a analizar:
{texto}
 
Análisis:""",
    input_variables=["texto"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,  # Baja temperatura para respuestas más consistentes
)

chain = prompt | llm | parser

# Texto de ejemplo
texto_prueba = """
La nueva película de ciencia ficción 'Estrella Galáctica' es absolutamente 
espectacular. Los efectos visuales son impresionantes y la trama mantiene 
la tensión durante toda la película. Los actores principales entregan 
actuaciones convincentes que realmente te hacen creer en este mundo futurista.
Sin duda una de las mejores películas del año.
"""
 
try:
    # Invocar la cadena
    resultado = chain.invoke({"texto": texto_prueba})
    
    # Acceder a los datos
    print("=== RESULTADO DEL ANÁLISIS ===")
    print(f"Resumen: {resultado.resumen}")
    print(f"Sentimiento: {resultado.sentimiento}")
    print(f"Palabras clave: {', '.join(resultado.palabras_clave)}")
    
    # Exportar como JSON
    print("\n=== JSON RESULTANTE ===")
    print(resultado.model_dump_json(indent=2))
    
    # Exportar como diccionario
    dict_resultado = resultado.model_dump()
    print(f"\nTipo de objeto: {type(resultado)}")
    print(f"Tipo de diccionario: {type(dict_resultado)}")
    
except Exception as e:
    print(f"❌ Error durante el procesamiento: {e}")

Instrucciones generadas:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"resumen": {"description": "Resumen breve del texto", "title": "Resumen", "type": "string"}, "sentimiento": {"description": "Sentimiento del texto (Positivo, neutro o negativo)", "title": "Sentimiento", "type": "string"}, "palabras_clave": {"description": "Lista de palabras clave del texto", "items": {"type": "string"}, "title": "Palabras Clave", "type": "array"}}, "required": ["resumen", "sentimiento", "palabras_clave"]}
```
=== RESULTADO DEL ANÁLISIS ===
Resumen: La película de ciencia ficción 'Estrel